Chapter 5 Running Time Analysis<br><br>
The big-O notation gives a very convenient way of grouping this (running time) functions into classes that can easily be compared.<br>
The goal of increasing efficiency in our data structures will be the primary motivation for introducing new structures and new ideas as we proceed.

5.1 Timing Programs

In [1]:
def duplicates1(L):
    n = len(L)
    for i in range(n):
        for j in range(n):
            if i != j and L[i] == L[j]:
                return True
    return False
assert(duplicates1([1,2,6,3,4,5,6,7,8]))
assert(not duplicates1([1,2,3,4]))

This is a function that takes a list as input, it returns True if there are any duplicates and False otherwise.

A basic question that we will ask again and again is the following?<br>
How fast is this code?<br>
The simplest answer to this question comes from simply running the program and measuring how long it takes.

In [2]:
import time
for i in range(5):
    n = 1000
    start = time.time()
    duplicates1(list(range(n)))
    timetaken = time.time() - start
    print("Time taken for n = ", n, ": ", timetaken)

Time taken for n =  1000 :  0.28138184547424316
Time taken for n =  1000 :  0.06571292877197266
Time taken for n =  1000 :  0.05651402473449707
Time taken for n =  1000 :  0.05131340026855469
Time taken for n =  1000 :  0.05218935012817383


Notice that we see some variation in the time required. This is caused by many factors, but the main one is that the computer is performing many other tasks at the same time. It is running an operating system and several other programs at the same time.<br>
Let’s run the code several times and take the average to smooth them out for a given computer.

In [3]:
import time

def timetrials(func, n, trials=10):
    totaltime=0
    for i in range(trials):
        start=time.time()
        func(list(range(n)))
        totaltime+=time.time() - start
    print("average =%10.7f for n = %d" % (totaltime/trials, n))

for n in [50, 100, 200, 400, 800, 1600, 3200]:
    timetrials(duplicates1, n)

average = 0.0007489 for n = 50
average = 0.0029428 for n = 100
average = 0.0100616 for n = 200
average = 0.0234177 for n = 400
average = 0.0583730 for n = 800
average = 0.1466960 for n = 1600
average = 0.5970231 for n = 3200


We can now look at the average running time as the length of the list gets longer, the average time goes up as the length n increases.

To make our code faster. Simple improvements can be made by eliminating situations where we are doing unnecessary or redundant work.<br>
In the duplicates1 function, we are comparing each pair of elements twice because both i and j range over all n indices. We can eliminate this using a standard trick of only letting j range up to i.

In [4]:
def duplicates2(L):
    n=len(L)
    for i in range(1, n):
        for j in range(i):
            if L[i] == L[j]:
                return True
    return False

for n in [50, 100, 200, 400, 800, 1600, 3200]:
    timetrials(duplicates2, n)

average = 0.0002569 for n = 50
average = 0.0010517 for n = 100
average = 0.0040986 for n = 200
average = 0.0181228 for n = 400
average = 0.0172263 for n = 800
average = 0.0554326 for n = 1600
average = 0.2541026 for n = 3200


In [6]:
#shorter version of duplicates2

def duplicates3(L):
    return any(L[i] == L[j] for i in range(1, len(L)) for j in range(i))

for n in [50, 100, 200, 400, 800, 1600, 3200]:
    timetrials(duplicates3, n)

average = 0.0004117 for n = 50
average = 0.0022580 for n = 100
average = 0.0058756 for n = 200
average = 0.0155595 for n = 400
average = 0.0187890 for n = 800
average = 0.0738902 for n = 1600
average = 0.2993898 for n = 3200


This last optimization reduces the number of lines of code and may be desirable for code readability, but it doesn’t improve the speed.

In [7]:
def duplicates4(L):
    n = len(L)
    L.sort()
    for i in range(n-1):
        if L[i] == L[i+1]:
            return True
    return False

In [8]:
def duplicates5(L):
    n = len(L)
    L.sort()
    return any(L[i] == L[i+1] for i in range(n-1))

In [9]:
def duplicates6(L):
    s = set()
    for e in L:
        if e in s:
            return True
        s.add(e)
    return False

In [10]:
def duplicates7(L):
    return len(L) != len(set(L))

In [11]:
def duplicates8(L):
    s = set()
    return any(e in s or s.add(e) for e in L)

In [12]:
for n in [50, 100, 200, 400, 800, 1600, 3200]:
    print("Quadratic: ", end="")
    timetrials(duplicates3, n)
    print("Sorting:", end="")
    timetrials(duplicates5, n)
    print("Sets:", end="")
    timetrials(duplicates7, n)
    print('---------------------------')

Quadratic: average = 0.0000739 for n = 50
Sorting:average = 0.0000047 for n = 50
Sets:average = 0.0000014 for n = 50
---------------------------
Quadratic: average = 0.0003143 for n = 100
Sorting:average = 0.0000107 for n = 100
Sets:average = 0.0000026 for n = 100
---------------------------
Quadratic: average = 0.0013366 for n = 200
Sorting:average = 0.0000234 for n = 200
Sets:average = 0.0000064 for n = 200
---------------------------
Quadratic: average = 0.0101454 for n = 400
Sorting:average = 0.0000907 for n = 400
Sets:average = 0.0000165 for n = 400
---------------------------
Quadratic: average = 0.0229714 for n = 800
Sorting:average = 0.0000775 for n = 800
Sets:average = 0.0000144 for n = 800
---------------------------
Quadratic: average = 0.0783082 for n = 1600
Sorting:average = 0.0001675 for n = 1600
Sets:average = 0.0000369 for n = 1600
---------------------------
Quadratic: average = 0.3359103 for n = 3200
Sorting:average = 0.0002696 for n = 3200
Sets:average = 0.0000545 fo

5.2 Example: Adding the first k numbers.<br><br>
A program that adds up the first k positive integers and returns both the sum and time required to do the computation.

In [1]:
import time
def sumk(k):
    start = time.time()
    total = 0
    for i in range(k+1):
        total = total + i
        end = time.time()
    return total, end-start

for i in range(5):
    print("Sum: %d, time taken: %f" % sumk(10000))

Sum: 50005000, time taken: 0.004376
Sum: 50005000, time taken: 0.003677
Sum: 50005000, time taken: 0.003587
Sum: 50005000, time taken: 0.003650
Sum: 50005000, time taken: 0.003462


In [3]:
def timetrials(func, k, trials = 10):
    totaltime = 0
    for i in range(trials):
        totaltime += func(k)[1]
    print("average =%10.7f for k = %d" % (totaltime/trials, k))

timetrials(sumk, 10000)
timetrials(sumk, 100000)
timetrials(sumk, 1000000)
timetrials(sumk, 10000000)

average = 0.0083256 for k = 10000
average = 0.0345149 for k = 100000
average = 0.1135713 for k = 1000000
average = 1.1836195 for k = 10000000


Seeing the times for different values of k reveals a rather unsuprising pattern. As k goes up by a factor of 10, the time required for sumk also goes up by a factor of 10.<br>
Tere is another, much simpler way to compute the sum of the numbers from 1 to k using a formula that is very important.
$$
\sum_{i=1}^{k} i = 1+2+3+...+k = k(k+1)/2
$$

It suffices to observe that you can add the numbers in pairs, matching i with k − i + 1 starting with 1 and k. There are k/2 such pairs and each adds up to k + 1. Let’s use this formula to rewrite our sumk function and time it.

In [4]:
import time

def sumk2(k):
    start = time.time()
    total = (k*(k+1)//2)
    end = time.time()
    return total, end - start

timetrials(sumk2, 10000)
timetrials(sumk2, 100000)
timetrials(sumk2, 1000000)
timetrials(sumk2, 10000000)
timetrials(sumk2, 100000000)

average = 0.0000002 for k = 10000
average = 0.0000003 for k = 100000
average = 0.0000003 for k = 1000000
average = 0.0000003 for k = 10000000
average = 0.0000003 for k = 100000000


This is much much faster. Even as k becomes very large, it doesn’t slow down.

5.3 Modeling the Running Time of a Program<br><br>
Introducing a general technique for describing and summarizing the number of operations required to run a piece of code, be it a single line, a function, or an entire program.It’s not enough to count lines of code. A single line of code can do a lot of stuff.

In [5]:
def f001(k):
    return [sum([i, i + 1] * 100) for i in range(k)]
print(f001(9))

[100, 300, 500, 700, 900, 1100, 1300, 1500, 1700]


Atomic operations include - arithmetic and boolean operations - variable assignment - accessing the value of a variable from its name - branching if/for/while statements - calling a function - returning from a function.

5.3.1 List Operations


| Operation Name             |        Code        |        Cost        |
|:---------------------------|:------------------:|-------------------:|
| index access               | L[ i ]             |         1          |
| index assignment           | L[ i ] = newvalue  |         1          |
| Append                     | L.append(newitem)  |         1          |
|Pop (from end of list)      | L.pop()            | 1
|Pop (from index i)          | L.pop(i)           | n−i
|Insert at index i           | insert(i, newitem) | n−i
|Delete an item (at index i) | del(item)          | n−i
|Membership testing          | item in L          | n
|Slice                       | L[a:b]             | b−a
|Concatenate two lists       | L1 + L2            | n1 + n2
|Sort                        | L.sort()           | n log2 n

These running times are the same for the other sequential col- lections, list and str assuming the operation exists for those immutable types. For example, index access, membership testing, slicing, and concatenation all work.

5.3.2 Dictionary Operations

| Operation Name             |        Code        |        Cost        |
|:---------------------------|:------------------:|-------------------:|
| Get item                   | D[ key ]           |         1          |
| Set item                   | D[ key ] = value   |         1          |
| (key) membership testing   | key in D           |         1          |
| Delete an item by its key  | del D[ key ]       |         1          |

Unlike the list operations, the costs of dict operations are a bit mysterious. Some may seem downright impossible. Should it really cost just one atomic operation to test if a given item is in a set of a billion elements?<br>
There are three things to remember: 
1. We will study how dictionaries are implemented and how they exploit one of the wonderful, clever ideas of computer science. 
2. This is just a model, albeit a useful and accurate one.
3. The actual cost is a kind of average. It could take longer sometimes.

5.3.3 Set Operations<br><br>
A set is very much like a dict where the entries have keys but no values. They are implemented the same way and so, the running times for their common operations are the same. Some set operations that correspond to the mathematical idea of a set do not correspond to operations on dictionaries. Operations that produce a new set will leave the input sets unchanged. Below, let nA be the size of set A and let nB be the size of set B.

| Operation Name        |        Code        |        Cost                        |
|:----------------------|:------------------:|-----------------------------------:|
| Add a new item        | A.add(newitem)     |                1                   |
| Delete an item        | A.delete(item)     |                1                   |
| Union                 | A \| B             |*n<sub>A</sub>* + *n<sub>B</sub>*   |
| Intersection          | A & B              |*min n<sub>A</sub>*, *n<sub>B</sub>*|
| Set differences       | A - B              |        *n<sub>A</sub>*             |
| Symmetric difference  | A ∧ B              |*n<sub>A</sub>* + *n<sub>B</sub>*   |


5.4 Asymptotic Analysis and the Order of Growth<br><br>
The goal is not to predict exactly how much time an algorithm will take, but rather to predict the order of growth of the time as the input size grows.<br>
If we have algorithm that operates on a list of length n, the running time could be proportional to n. In that case, the algorithm will take 100 times longer on a list that is 100 times longer. A second algorithm might have a running time proportional to n2 for inputs of length n. Then, the algorithm will take 10000 times longer on a list that is 100 times longer.<br><br>
The size of the input refers to the number of bits needed to encode it. As we will be ignoring constant factors, we could just as easily refer to the number of words (a word is 64 bits) needed to encode it. An integer or a float is generally stored in one word.<br>Technically, we can store some really big numbers in a Python integer which would require many more words, but as a convention, we will assume that ints and floats fit in a constant number of bits. This is necessary to assume that arithmetic takes constant time.

5.5 Focus on the Worst Case<br><br>
Usually, different inputs of the same length may have different running times.<br>
The standard convention we will use most of the time is to consider the worst case. We are looking for upper bounds on the running time.<br>
If the algorithm has a running time that is better than the analysis predicts, that’s okay.

5.6 Big-O<br><br>

The formal mathematical definition that allows us to ignore constant factors is called the **big-O notation.**<br>
Example:  
$$
f(n) = O(n^2)
$$

if there exists a constant c such that<br>
*f (n) ≤ cn<sup>2</sup>* for all sufficiently large n.<br>
This is correct for *f (n) = 5n<sup>2</sup> + 3n + 2* because if we take *c = 6*, we see that as long as *n > 4*, we have
$$
f (n) = 5n^2 + 3n + 2 < 5n^2 + 4n < 5n^2 + n^2 ≤ 6n^2
$$

The formal definition of the big-O notation:<br>
Given (nondecreasing) functions ***f*** and ***g***, we say ***f (n) = O(g(n))*** if there exist constants ***c*** and ***n<sub>0</sub>*** such that for all ***n > n<sub>0</sub>*** we have ***f (n) ≤ cg(n).***

5.7 The most important features of big-O usage<br><br>

1. The big-O hides constant factors. Any term that does not depend on the size of the input is considered a constant and will be suppressed in the big-O.
2. The big-O tells us about what will eventually be true when the input is sufficiently large.

5.8 Practical Use of the Big-O and Common Functions

Even though the definition of the big-O notation allows us to compare all kinds of functions, we will usually use it to simplify functions, eliminating extraneous constants and low order terms.<br>
So, for example, you should write ***O(n)*** instead of ***O(3n)*** and ***O(n<sup>2</sup>)*** instead of ***O(5n<sup>2</sup> + 3n + 2).***<br>
There are several functions that will come up so often that we will want to have them in our vocabulary.<br>

- Constant Functions    ------->***O(1)***
- Logarithmic Functions ------->***O(log n)***
- Linear Functions      ------->***O(n)***
- n Log n               ------->***O(n log n)***
- Quadratic Functions   ------->***O(n<sup>2</sup>)***
- Polynomial Functions  ------->***O(n<sup>k</sup>)*** for some constant k.
- Exponential Functions ------->***O(2<sup>n</sup>)*** (this is different from 2<sup>O(n)</sup>)
- Factorial Functions   ------->***O(n!)***

5.9 Bases for Logarithms<br><br>
You may have noticed that we didn’t give a base for the logarithm above.<br>
The reason is that inside the big-O, logarithms of any constant base are the same.<br>
That is, ***log<sub>a</sub> (n) = O(log<sub>b</sub> (n))***, where a and b are any two constants. Here’s the proof.<br>
$$
Let\ c\ = \ \frac{1}{log_{b}(a)}\ and\ n_{0} = 0
$$

$$
log_{a}(n)\ = \ \frac{log_{b}(n)}{log_{b}(a)} \leq clog_{b}(n) \ for \ all \ n > n_{0} 
$$

5.10 Practice examples<br><br>
In each of the following examples, assume the input is a list of length n.

In [8]:
def f002(L):
    newList = []        # 2
    for i in L:         # loop n times
        if i % 2 == 0:  # 1
            newList.append(i)   # 1
    return newList      # 1

Let’s count up the cost of each line of code. The costs are in the comments. So, the total cost is something like ***2n + 3*** in the worst case (i.e. when all the items are even). We would report this as ***O(n)*** and we would call this a linear-time algorithm, or sometimes simply a linear algorithm.

In [6]:
def f003(L):
    x = 0           # 1
    for i in L:     # loop n times
        for j in L: # loop n times
            x += i*j    # 3
    return x        # 1

Again, let’s count up the cost of each line of code. The costs are in the comments. The inner loop costs ***3n*** and it runs n times, so the total for the whole method is ***3n<sup>2</sup> + 2.*** We would report this as ***O(n<sup>2</sup> )*** and call this a quadratic-time algorithm, or sometimes simply a quadratic algorithm.

In [9]:
def f004(L):
    x = 0           # 1
    for j in range(1, len(L)):  # loop n-1 times
        for i in range(j):      # loop j times
            x += L[i] * L[j]    # 5
    return x        # 1

The first costs 5, the second costs 10, and so on to that the *j* th costs 5*j*. The total costs including initializing x and returning is
$$
2 \ + \ \sum_{i=1}^{n-1} \ 5i \ = \ 2+5 \ \sum_{i=1}^{n-1} \ = \ 2+ \frac{5n(n-1)}{2} \ = \ O(n^2)
$$